# Dev Area for the forecaster in the Sonntagsfrage project

## Definitions and initialisations

In [3]:
# imports
import pyodbc
import datetime as dt
import os
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor

In [4]:
# default params
server = 'sonntagsfrage-server.database.windows.net'
database = 'sonntagsfrage-sql-db'
username = 'sonntagspredictor'
password = 'das_ist_EiN_sICHe_res_PasSworD_!' 
driver = '{ODBC Driver 17 for SQL Server}'
path_dataframes = '..\\..\\dataframes\\'
target_cols = ['CDU_CSU', 'SPD', 'GRUENE', 'FDP', 'LINKE', 'PIRATEN', 'AfD', 'Linke_PDS', 'PDS', 'REP_DVU', 'Sonstige']
date_col = 'Datum_dt'

In [5]:
# open connection
conn_str = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password
print(conn_str)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

DRIVER={ODBC Driver 17 for SQL Server};SERVER=sonntagsfrage-server.database.windows.net;PORT=1433;DATABASE=sonntagsfrage-sql-db;UID=sonntagspredictor;PWD=das_ist_EiN_sICHe_res_PasSworD_!


ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open server 'sonntagsfrage-server' requested by the login. Client with IP address '77.0.186.224' is not allowed to access the server.  To enable access, use the Windows Azure Management Portal or run sp_set_firewall_rule on the master database to create a firewall rule for this IP address or address range.  It may take up to five minutes for this change to take effect. (40615) (SQLDriverConnect); [42000] [Microsoft][ODBC Driver 17 for SQL Server]Ungültiges Attribut für die Verbindungszeichenfolge (0); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open server 'sonntagsfrage-server' requested by the login. Client with IP address '77.0.186.224' is not allowed to access the server.  To enable access, use the Windows Azure Management Portal or run sp_set_firewall_rule on the master database to create a firewall rule for this IP address or address range.  It may take up to

In [8]:
def load_df_from_file(filename, path=path_dataframes):
    """
        Loads a local file into a Pandas Dataframe.

        :param filename: Name of the file that shall be loaded.
        :param path: Path to the file.
        :return: pandas.Dataframe: Returns a Pandas Dataframe created from the loaded file.
    """
#    os.chdir('..\\..\\dataframes')
#    os.getcwd()
    df = pd.read_pickle(f"{path}/{filename}.pkl")
    return df

## Test SQL capabilites from within python

### 1. Can tables be created and recreated from python?

In [5]:
sqlstmt = """truncate table sonntagsfrage.results_questionaire_clean"""
cursor.execute(sqlstmt)
conn.commit()


OperationalError: ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]TCP-Anbieter: Eine vorhandene Verbindung wurde vom Remotehost geschlossen.\r\n (10054) (SQLExecDirectW); [08S01] [Microsoft][ODBC Driver 17 for SQL Server]Kommunikationsverbindungsfehler (10054)')

In [ ]:
;
insert into sonntagsfrage.results_questionaire_clean
select  
Datum
,cast( coalesce( '0', replace(isnull(CDU_CSU  , '0'), '-', '0')) as numeric) CDU_CSU
,cast( coalesce( '0', replace(isnull(SPD      , '0'), '-', '0')) as numeric) SPD          
,cast( coalesce( '0', replace(isnull(GRUENE   , '0'), '-', '0')) as numeric) GRUENE      
,cast( coalesce( '0', replace(isnull(FDP      , '0'), '-', '0')) as numeric) FDP        
,cast( coalesce( '0', replace(isnull(LINKE    , '0'), '-', '0')) as numeric) LINKE        
,cast( coalesce( '0', replace(isnull(PIRATEN  , '0'), '-', '0')) as numeric) PIRATEN      
,cast( coalesce( '0', replace(isnull(AfD      , '0'), '-', '0')) as numeric) AfD       
,cast( coalesce( '0', replace(isnull(Linke_PDS, '0'), '-', '0')) as numeric) Linke_PDS    
,cast( coalesce( '0', replace(isnull(PDS      , '0'), '-', '0')) as numeric) PDS          
,cast( coalesce( '0', replace(isnull(REP_DVU  , '0'), '-', '0')) as numeric) REP_DVU  
, cast( coalesce( '0', replace(replace(replace( isnull(Sonstige , '0'), '-', '0'), 'PIR', '0'), 'WASG3', '0')) as numeric) Sonstige
,Befragte   
,Zeitraum
,''
from sonntagsfrage.results_questionaire q
GO

## 2. Working area with dataframes from intermediate steps

In [40]:
df_feat = load_df_from_file('load_data_df_survey_results')

In [41]:
df_feat.columns

Index(['Datum', 'CDU_CSU', 'SPD', 'GRUENE', 'FDP', 'LINKE', 'PIRATEN', 'AfD',
       'Linke_PDS', 'PDS', 'REP_DVU', 'Sonstige', 'Befragte', 'Zeitraum',
       'meta_insert_ts'],
      dtype='object')

In [42]:
df_feat.dtypes

Datum              object
CDU_CSU           float64
SPD               float64
GRUENE            float64
FDP               float64
LINKE             float64
PIRATEN           float64
AfD               float64
Linke_PDS         float64
PDS               float64
REP_DVU           float64
Sonstige          float64
Befragte           object
Zeitraum           object
meta_insert_ts     object
dtype: object

In [44]:
df_feat.head()

,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts
0,22.12.2000,37.0,39.0,7.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,,,
1,15.12.2000,37.0,39.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,
2,08.12.2000,36.0,39.0,8.0,8.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,
3,01.12.2000,37.0,38.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,
4,24.11.2000,37.0,39.0,7.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,


In [39]:
df_feat.dtypes

Datum                     object
CDU_CSU                  float64
SPD                      float64
GRUENE                   float64
FDP                      float64
LINKE                    float64
PIRATEN                  float64
AfD                      float64
Linke_PDS                float64
PDS                      float64
REP_DVU                  float64
Sonstige                 float64
Befragte                  object
Zeitraum                  object
meta_insert_ts            object
Datum1                    object
Datum2                    object
Datum3            datetime64[ns]
dtype: object

### Check how functions around the index work

In [144]:
df_data = load_df_from_file('load_data_df_all_data_combined')
df_survey_data = load_df_from_file('load_survey_data')
df_feat = load_df_from_file('generate_features_all_features')
df_feat.head()

,Datum_dt,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,...,day_in_month_sin,day_in_month_cos,dayofyear_sin,dayofyear_cos,weekday_sin,weekday_cos,calendar_week_sin,calendar_week_cos,month_sin,month_cos
0,2000-12-22,22.12.2000,37.0,39.0,7.0,6.0,0.0,0.0,0.0,0.0,...,-9.945219e-01,-0.104528,-0.137279,0.990532,-0.433884,-0.900969,-0.120537,0.992709,-2.449294e-16,1.000000
1,2000-12-15,15.12.2000,37.0,39.0,8.0,7.0,0.0,0.0,0.0,0.0,...,5.665539e-16,-1.000000,-0.255353,0.966848,-0.433884,-0.900969,-0.239316,0.970942,-2.449294e-16,1.000000
2,2000-08-12,08.12.2000,36.0,39.0,8.0,8.0,0.0,0.0,0.0,0.0,...,5.877853e-01,-0.809017,-0.668064,-0.744104,-0.974928,-0.222521,-0.663123,-0.748511,-8.660254e-01,-0.500000
3,2000-01-12,01.12.2000,37.0,38.0,8.0,7.0,0.0,0.0,0.0,0.0,...,5.877853e-01,-0.809017,0.205104,0.978740,0.974928,-0.222521,0.239316,0.970942,5.000000e-01,0.866025
4,2000-11-24,24.11.2000,37.0,39.0,7.0,7.0,0.0,0.0,0.0,0.0,...,-9.510565e-01,0.309017,-0.580800,0.814046,-0.433884,-0.900969,-0.568065,0.822984,-5.000000e-01,0.866025


In [83]:
df_test = df_data
df_test['Datum_dt'] = pd.to_datetime(df_test['Datum_dt'], format='%d.%m.%Y')
df_test.head()

,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts,Datum_dt
0,22.12.2000,37.0,39.0,7.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,,,,2000-12-22
1,15.12.2000,37.0,39.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,,2000-12-15
2,08.12.2000,36.0,39.0,8.0,8.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,,2000-08-12
3,01.12.2000,37.0,38.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,,2000-01-12
4,24.11.2000,37.0,39.0,7.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,,2000-11-24


In [86]:
df_feat_check_idx = df_data
df_feat_check_idx = df_feat_check_idx.set_index('Datum_dt')

In [90]:
# dtype returns a string containing the name of the type
print(df_feat.index.dtype == 'int64')
print(df_feat_check_idx.index.dtype == 'datetime64[ns]')

True
True


### Hwo to get max Date, min Date, all Dates etc. from datetime index

In [158]:
df_data = load_df_from_file('load_data_df_all_data_combined')
df_feat = load_df_from_file('generate_features_all_features')
df_data_idx = df_data.set_index('Datum_dt')
df_feat_idx = df_feat.set_index('Datum_dt')

In [159]:
print(df_data_idx.index.min())

1998-01-08 00:00:00


In [167]:
df_data_idx.index = df_data_idx.index.sort_values()
dates_sorted = df_data_idx.index.unique()

In [263]:
df_test = df_data_idx

for date in dates_sorted[1:]:
    df_test.loc[df_test.index == date, 'test'] = date
    ret = df_test[df_test.index == date]
#     ret = df_data_idx.index.isin(date)
    print(date)
    print(ret)

1998-02-05 00:00:00
                 Datum  CDU_CSU   SPD  GRUENE  FDP  LINKE  PIRATEN  AfD  \
Datum_dt                                                                  
1998-02-05  02.05.1998     35.0  45.0     6.0  5.0    0.0      0.0  0.0   

            Linke_PDS  PDS  REP_DVU  Sonstige Befragte Zeitraum  \
Datum_dt                                                          
1998-02-05        0.0  4.0      4.0       1.0                     

           meta_insert_ts       test  
Datum_dt                              
1998-02-05                1998-02-05  
1998-02-10 00:00:00
                 Datum  CDU_CSU   SPD  GRUENE  FDP  LINKE  PIRATEN  AfD  \
Datum_dt                                                                  
1998-02-10  02.10.1998     36.0  41.0     7.0  6.0    0.0      0.0  0.0   

            Linke_PDS  PDS  REP_DVU  Sonstige Befragte Zeitraum  \
Datum_dt                                                          
1998-02-10        0.0  5.0      3.0       2.0          

In [172]:
df_test.head()

,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts,test
Datum_dt,,,,,,,,,,,,,,,,
1998-01-08,22.12.2000,37.0,39.0,7.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,,,,NaT
1998-02-05,15.12.2000,37.0,39.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,,1998-02-05
1998-02-10,08.12.2000,36.0,39.0,8.0,8.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,,1998-02-10
1998-04-04,01.12.2000,37.0,38.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,,1998-04-04
1998-04-07,24.11.2000,37.0,39.0,7.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,,1998-04-07


In [173]:
a= ['bla', 'test'] + ['aha']
print(a)
print(a[1:-1])

['bla', 'test', 'aha']
['test']


In [111]:
df_data_idx.dtypes

Datum              object
CDU_CSU           float64
SPD               float64
GRUENE            float64
FDP               float64
LINKE             float64
PIRATEN           float64
AfD               float64
Linke_PDS         float64
PDS               float64
REP_DVU           float64
Sonstige          float64
Befragte           object
Zeitraum           object
meta_insert_ts     object
dtype: object

### Insert a new row into a dataframe

In [231]:
# easy example
a_row = pd.Series([1, 2])
df = pd.DataFrame([[3, 4], [5, 6]])

row_df = pd.DataFrame([a_row])
df = pd.concat([row_df, df], ignore_index=True)

print(row_df)
# print(df)

   0  1
0  1  2


In [245]:
# load data and set variables
date = '15.11.2020'
df_data = load_df_from_file('load_data_df_all_data_combined')
df_data.head()

,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts,Datum_dt
0,22.12.2000,37.0,39.0,7.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,,,,2000-12-22
1,15.12.2000,37.0,39.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,,2000-12-15
2,08.12.2000,36.0,39.0,8.0,8.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,,2000-08-12
3,01.12.2000,37.0,38.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,,2000-01-12
4,24.11.2000,37.0,39.0,7.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,,2000-11-24


In [232]:
zeroes = np.zeros(len(df_data.columns.values))
zeroes_series = pd.Series(zeroes)
print(zeroes)
zeroes_series[-1] = date
zeroes_series.index = zeroes_series.index + 1
zeroes_series = zeroes_series.sort_index()
zeroes_series = zeroes_series
row_df = pd.DataFrame([zeroes_series])
print(row_df)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
           0    1    2    3    4    5    6    7    8    9    10   11   12  \
0  15.11.2020  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

    13   14   15   16  
0  0.0  0.0  0.0  0.0  


In [236]:
df_data_plus = pd.concat([df_data, row_df], ignore_index=True)
df_data_plus.head()

,0,1,2,3,4,5,6,7,8,9,...,GRUENE,LINKE,Linke_PDS,PDS,PIRATEN,REP_DVU,SPD,Sonstige,Zeitraum,meta_insert_ts
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,0.0,0.0,6.0,0.0,0.0,39.0,5.0,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,0.0,0.0,5.0,0.0,0.0,39.0,4.0,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,0.0,0.0,5.0,0.0,0.0,39.0,4.0,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,0.0,0.0,5.0,0.0,0.0,38.0,5.0,,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,0.0,0.0,5.0,0.0,0.0,39.0,5.0,,


In [251]:
df_data.loc[-1] = zeroes_series

In [253]:
df_data[df_data.Datum == date].head()

,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts,Datum_dt


In [212]:
a_row = pd.Series([1, 2])
print(a_row)

0    1
1    2
dtype: int64


In [ ]:
a_row = pd.Series([1, 2])
df = pd.DataFrame([[3, 4], [5, 6]])

row_df = pd.DataFrame([a_row])
df = pd.concat([row_df, df], ignore_index=True)


### Get model Training to work

In [255]:
df_feat = load_df_from_file('generate_features_all_features')
df_feat_idx = df_feat.set_index('Datum_dt')
df_for_train = df_feat_idx

In [257]:
df_for_train[df_for_train.isnull().any(axis=1)]

,day_in_month_sin,calendar_week_sin,weekday_sin,dayofyear_sin,month_sin,day_in_month_cos,calendar_week_cos,weekday_cos,dayofyear_cos,month_cos,...,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige
Datum_dt,,,,,,,,,,,,,,,,,,,,,


In [256]:
X_train = df_for_train.drop(columns=target_cols, axis=1)#.select_dtypes(['number'])
print(X_train)

            day_in_month_sin  calendar_week_sin  weekday_sin  dayofyear_sin  \
Datum_dt                                                                      
1998-01-08      9.945219e-01           0.239316     0.433884       0.137279   
1998-02-05      8.660254e-01           0.663123     0.433884       0.580800   
1998-02-10      8.660254e-01           0.748511     0.781831       0.648630   
1998-04-04      7.431448e-01           0.992709    -0.974928       0.998880   
1998-04-07      9.945219e-01           0.970942     0.781831       0.995105   
...                      ...                ...          ...            ...   
2020-08-20     -8.660254e-01          -0.822984     0.433884      -0.763889   
2020-09-01      2.079117e-01          -0.935016     0.781831      -0.880012   
2020-09-17     -4.067366e-01          -0.992709     0.433884      -0.976011   
2020-10-11      7.431448e-01          -0.970942    -0.781831      -0.981306   
2020-10-15      5.665539e-16          -0.935016     

In [155]:
y_train = df_feat_idx[target_cols]
print(y_train)

            CDU_CSU   SPD  GRUENE  FDP  LINKE  PIRATEN  AfD  Linke_PDS  PDS  \
Datum_dt                                                                      
2000-12-22     37.0  39.0     7.0  6.0    0.0      0.0  0.0        0.0  6.0   
2000-12-15     37.0  39.0     8.0  7.0    0.0      0.0  0.0        0.0  5.0   
2000-08-12     36.0  39.0     8.0  8.0    0.0      0.0  0.0        0.0  5.0   
2000-01-12     37.0  38.0     8.0  7.0    0.0      0.0  0.0        0.0  5.0   
2000-11-24     37.0  39.0     7.0  7.0    0.0      0.0  0.0        0.0  5.0   
...             ...   ...     ...  ...    ...      ...  ...        ...  ...   
2006-02-02     40.0  32.0     7.0  8.0    0.0      0.0  0.0        9.0  0.0   
2006-01-26     40.0  33.0     8.0  7.0    0.0      0.0  0.0        9.0  0.0   
2006-01-19     39.0  32.0     9.0  8.0    0.0      0.0  0.0        9.0  0.0   
2006-12-01     39.0  32.0     9.0  8.0    0.0      0.0  0.0        9.0  0.0   
2006-05-01     39.0  31.0     9.0  9.0    0.0      0

In [156]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

DecisionTreeRegressor()

In [254]:
model.predict(X_train)

array([[37. , 39. ,  7. , ...,  6. ,  0. ,  5. ],
       [37. , 39. ,  8. , ...,  5. ,  0. ,  4. ],
       [36. , 39. ,  8. , ...,  5. ,  0. ,  4. ],
       ...,
       [37.5, 30.5, 12. , ...,  0. ,  0. ,  3.5],
       [39. , 32. ,  9. , ...,  0. ,  0. ,  3. ],
       [38. , 25.5,  9. , ...,  0. ,  0. ,  4. ]])

### Debug Area

In [258]:
df_debug = load_df_from_file('df_debug')
df_debug.head()

,day_in_month_sin,calendar_week_sin,weekday_sin,dayofyear_sin,month_sin,day_in_month_cos,calendar_week_cos,weekday_cos,dayofyear_cos,month_cos,...,SPD_pred,GRUENE_pred,FDP_pred,LINKE_pred,PIRATEN_pred,AfD_pred,Linke_PDS_pred,PDS_pred,REP_DVU_pred,Sonstige_pred
Datum_dt,,,,,,,,,,,,,,,,,,,,,
1998-01-08,0.994522,0.239316,0.433884,0.137279,0.500000,-0.104528,0.970942,-0.900969,0.990532,0.866025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-05,0.866025,0.663123,0.433884,0.580800,0.866025,0.500000,0.748511,-0.900969,0.814046,0.500000,...,43.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0


### Check predictions

In [178]:
df_preds = load_df_from_file('generate_predictions_finish')

In [179]:
pd.set_option('display.max_columns', None)
df_preds.head()

,day_in_month_sin,calendar_week_sin,weekday_sin,dayofyear_sin,month_sin,day_in_month_cos,calendar_week_cos,weekday_cos,dayofyear_cos,month_cos,nb_days_since_last_survey,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,CDU_CSU_pred,SPD_pred,GRUENE_pred,FDP_pred,LINKE_pred,PIRATEN_pred,AfD_pred,Linke_PDS_pred,PDS_pred,REP_DVU_pred,Sonstige_pred
Datum_dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1998-01-08,0.994522,0.239316,0.433884,0.137279,0.500000,-0.104528,0.970942,-0.900969,0.990532,0.866025,0,36.0,43.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-05,0.866025,0.663123,0.433884,0.580800,0.866025,0.500000,0.748511,-0.900969,0.814046,0.500000,28,35.0,45.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,4.0,1.0,36.0,43.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0
1998-02-10,0.866025,0.748511,0.781831,0.648630,0.866025,-0.500000,0.663123,0.623490,0.761104,0.500000,5,36.0,41.0,7.0,6.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,35.0,45.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,4.0,1.0
1998-04-04,0.743145,0.992709,-0.974928,0.998880,0.866025,0.669131,-0.120537,-0.222521,-0.047321,-0.500000,53,34.0,46.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,2.0,3.0,35.0,45.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,4.0,1.0
1998-04-07,0.994522,0.970942,0.781831,0.995105,0.866025,0.104528,-0.239316,0.623490,-0.098820,-0.500000,3,35.0,44.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,36.0,41.0,7.0,6.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0


In [180]:
df_preds[df_preds.index == df_preds.index.max()]

,day_in_month_sin,calendar_week_sin,weekday_sin,dayofyear_sin,month_sin,day_in_month_cos,calendar_week_cos,weekday_cos,dayofyear_cos,month_cos,nb_days_since_last_survey,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,CDU_CSU_pred,SPD_pred,GRUENE_pred,FDP_pred,LINKE_pred,PIRATEN_pred,AfD_pred,Linke_PDS_pred,PDS_pred,REP_DVU_pred,Sonstige_pred
Datum_dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-11-15,5.665539e-16,-0.663123,-0.781831,-0.699458,-0.5,-1.0,0.748511,0.62349,0.714673,0.866025,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,26.0,23.0,5.0,10.0,0.0,0.0,0.0,0.0,0.0,5.0


In [181]:
df_preds[df_preds.index > '2020-01-01']

,day_in_month_sin,calendar_week_sin,weekday_sin,dayofyear_sin,month_sin,day_in_month_cos,calendar_week_cos,weekday_cos,dayofyear_cos,month_cos,nb_days_since_last_survey,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,CDU_CSU_pred,SPD_pred,GRUENE_pred,FDP_pred,LINKE_pred,PIRATEN_pred,AfD_pred,Linke_PDS_pred,PDS_pred,REP_DVU_pred,Sonstige_pred
Datum_dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-10,8.660254e-01,2.393157e-01,-0.433884,0.171293,5.000000e-01,-0.500000,0.970942,-0.900969,0.985220,8.660254e-01,29,35.0,15.0,21.0,6.0,8.0,0.0,10.0,0.0,0.0,0.0,5.0,40.5,26.5,11.0,6.0,4.5,0.0,3.0,0.0,3.0,0.0,5.5
2020-01-23,-9.945219e-01,4.647232e-01,0.433884,0.385663,5.000000e-01,0.104528,0.885456,-0.900969,0.922640,8.660254e-01,13,26.0,14.0,24.0,8.0,8.0,0.0,14.0,0.0,0.0,0.0,6.0,31.0,32.0,8.0,6.0,7.0,0.0,11.0,0.0,0.0,0.0,5.0
2020-02-04,7.431448e-01,6.631227e-01,0.781831,0.566702,8.660254e-01,0.669131,0.748511,0.623490,0.823923,5.000000e-01,12,34.0,16.0,22.0,5.0,7.0,0.0,10.0,0.0,0.0,0.0,6.0,34.0,26.0,11.0,15.0,11.0,0.0,0.0,0.0,0.0,0.0,3.0
2020-02-07,9.945219e-01,6.631227e-01,-0.433884,0.608477,8.660254e-01,0.104528,0.748511,-0.900969,0.793572,5.000000e-01,3,37.0,16.0,20.0,5.0,7.0,0.0,10.0,0.0,0.0,0.0,5.0,39.0,26.0,11.0,4.0,9.0,0.0,5.0,0.0,0.0,0.0,6.0
2020-02-13,4.067366e-01,7.485107e-01,0.433884,0.687053,8.660254e-01,-0.913545,0.663123,-0.900969,0.726608,5.000000e-01,6,26.0,16.0,22.0,7.0,9.0,0.0,14.0,0.0,0.0,0.0,6.0,37.0,22.0,12.0,15.0,9.0,0.0,0.0,0.0,0.0,0.0,5.0
2020-03-09,9.510565e-01,9.709418e-01,0.000000,0.927542,1.000000e+00,-0.309017,0.239316,1.000000,0.373720,6.123234e-17,25,36.0,17.0,19.0,6.0,7.0,0.0,10.0,0.0,0.0,0.0,5.0,37.0,21.0,8.0,9.0,10.0,0.0,11.0,0.0,0.0,0.0,4.0
2020-03-19,-7.431448e-01,9.927089e-01,0.433884,0.977848,1.000000e+00,-0.669131,0.120537,-0.900969,0.209315,6.123234e-17,10,30.0,14.0,23.0,7.0,9.0,0.0,10.0,0.0,0.0,0.0,7.0,36.0,31.0,11.0,10.0,8.0,0.0,0.0,0.0,0.0,0.0,4.0
2020-04-06,9.510565e-01,9.709418e-01,0.000000,0.995105,8.660254e-01,0.309017,-0.239316,1.000000,-0.098820,-5.000000e-01,18,38.0,15.0,19.0,6.0,8.0,0.0,9.0,0.0,0.0,0.0,5.0,40.0,25.0,10.0,6.0,9.0,0.0,5.0,0.0,0.0,0.0,5.0
2020-04-16,-2.079117e-01,9.350162e-01,0.433884,0.963471,8.660254e-01,-0.978148,-0.354605,-0.900969,-0.267814,-5.000000e-01,10,38.0,17.0,19.0,5.0,7.0,0.0,9.0,0.0,0.0,0.0,5.0,33.0,17.0,13.0,8.0,10.0,0.0,14.0,0.0,0.0,0.0,5.0


# Scratch Area

In [9]:
df_survey = load_df_from_file('load_survey_data')
df_survey_idx = df_survey.set_index('Datum_dt')

In [10]:
print(df_survey_idx.dtypes)
print(df_survey_idx.index.max())
df_survey_idx.head()

Datum              object
CDU_CSU           float64
SPD               float64
GRUENE            float64
FDP               float64
LINKE             float64
PIRATEN           float64
AfD               float64
Linke_PDS         float64
PDS               float64
REP_DVU           float64
Sonstige          float64
Befragte           object
Zeitraum           object
meta_insert_ts     object
dtype: object
2020-11-15 00:00:00


,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts
Datum_dt,,,,,,,,,,,,,,,
2000-12-22,22.12.2000,37.0,39.0,7.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,,,
2000-12-15,15.12.2000,37.0,39.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,
2000-08-12,08.12.2000,36.0,39.0,8.0,8.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,,,
2000-01-12,01.12.2000,37.0,38.0,8.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,
2000-11-24,24.11.2000,37.0,39.0,7.0,7.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,,,


In [12]:
df_survey['test'] = min(df_survey['CDU_CSU'], 36)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [21]:
df_survey['max_val'] = 4
df_survey['PDS'] = df_survey[['PDS', 'max_val']].min(axis=1)

In [23]:
df_survey.drop('max_val', axis=1)

,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts,Datum_dt,test2
0,22.12.2000,36.0,39.0,7.0,6.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-12-22,36.0
1,15.12.2000,36.0,39.0,8.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,,,,2000-12-15,36.0
2,08.12.2000,36.0,39.0,8.0,8.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,,,,2000-08-12,36.0
3,01.12.2000,36.0,38.0,8.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-01-12,36.0
4,24.11.2000,36.0,39.0,7.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-11-24,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,26.01.2006,36.0,33.0,8.0,7.0,0.0,0.0,0.0,9.0,0.0,0.0,3.0,1000,24.01.–25.01.,,2006-01-26,36.0
811,19.01.2006,36.0,32.0,9.0,8.0,0.0,0.0,0.0,9.0,0.0,0.0,3.0,1000,17.01.–18.01.,,2006-01-19,36.0
812,12.01.2006,36.0,32.0,9.0,8.0,0.0,0.0,0.0,9.0,0.0,0.0,3.0,1000,10.01.–11.01.,,2006-12-01,36.0
813,05.01.2006,36.0,31.0,9.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,3.0,1672,02.01.–04.01.,,2006-05-01,36.0


In [24]:
df_survey.head(50)

,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts,Datum_dt,max_val,test2
0,22.12.2000,36.0,39.0,7.0,6.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-12-22,4,36.0
1,15.12.2000,36.0,39.0,8.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,,,,2000-12-15,4,36.0
2,08.12.2000,36.0,39.0,8.0,8.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,,,,2000-08-12,4,36.0
3,01.12.2000,36.0,38.0,8.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-01-12,4,36.0
4,24.11.2000,36.0,39.0,7.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-11-24,4,36.0
5,17.11.2000,36.0,39.0,7.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,,,,2000-11-17,4,36.0
6,10.11.2000,36.0,39.0,6.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-10-11,4,36.0
7,03.11.2000,36.0,39.0,6.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-03-11,4,36.0
8,27.10.2000,36.0,40.0,6.0,7.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-10-27,4,36.0
9,20.10.2000,36.0,39.0,6.0,8.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,,,,2000-10-20,4,36.0


In [50]:
df_survey_idx.count()

Datum             814
CDU_CSU           814
SPD               814
GRUENE            814
FDP               814
LINKE             814
PIRATEN           814
AfD               814
Linke_PDS         814
PDS               814
REP_DVU           814
Sonstige          814
Befragte          814
Zeitraum          814
meta_insert_ts    814
dtype: int64

In [64]:
for row in df_survey:
    print(df_survey.loc[0,:])

Datum                      22.12.2000
CDU_CSU                            37
SPD                                39
GRUENE                              7
FDP                                 6
LINKE                               0
PIRATEN                             0
AfD                                 0
Linke_PDS                           0
PDS                                 6
REP_DVU                             0
Sonstige                            5
Befragte                             
Zeitraum                             
meta_insert_ts                       
Datum_dt          2000-12-22 00:00:00
Name: 0, dtype: object
Datum                      22.12.2000
CDU_CSU                            37
SPD                                39
GRUENE                              7
FDP                                 6
LINKE                               0
PIRATEN                             0
AfD                                 0
Linke_PDS                           0
PDS                        

In [76]:
len(df_survey)

814

In [80]:
df_survey.astype(str).iloc[0,1:].str.cat(sep=',')

'37.0,39.0,7.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,,,,2000-12-22'

In [86]:
df_survey.astype(str).iloc[0,0]

'22.12.2000'

In [81]:
pred_cols = []

In [83]:
pred_cols.append('a')

In [87]:
pd.Series(pred_cols)

0    a
1    a
dtype: object

In [176]:
df_survey_2 = df_survey.copy()
df_survey_2['Datum_dt_bckp'] = df_survey_2['Datum_dt'].astype(str)
df_survey_2_idx = df_survey_2.set_index('Datum_dt')
df_survey_2_idx = df_survey_2_idx.sort_index()
df_survey_2_idx['last_survey_date'] = df_survey_2_idx['Datum_dt_bckp'].shift(1)
df_survey_2_idx

,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts,Datum_dt_bckp,last_survey_date
Datum_dt,,,,,,,,,,,,,,,,,
1998-01-08,01.08.1998,36.0,43.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,,,,1998-01-08,NaN
1998-02-05,02.05.1998,35.0,45.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,4.0,1.0,,,,1998-02-05,1998-01-08
1998-02-10,02.10.1998,36.0,41.0,7.0,6.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,,,,1998-02-10,1998-02-05
1998-04-04,04.04.1998,34.0,46.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,2.0,3.0,,,,1998-04-04,1998-02-10
1998-04-07,04.07.1998,35.0,44.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,,,,1998-04-07,1998-04-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-20,20.08.2020,37.0,16.0,17.0,6.0,8.0,0.0,10.0,0.0,0.0,0.0,6.0,1065,18.08.–19.08.,,2020-08-20,2020-07-23
2020-09-01,09.01.2020,27.0,13.0,23.0,9.0,8.0,0.0,14.0,0.0,0.0,0.0,6.0,1506,07.01.–08.01.,,2020-09-01,2020-08-20
2020-09-17,17.09.2020,36.0,16.0,20.0,5.0,8.0,0.0,10.0,0.0,0.0,0.0,5.0,1051,15.09.–16.09.,,2020-09-17,2020-09-01


In [177]:
df_min_date = df_survey_2_idx[df_survey_2_idx.index == df_survey_2_idx.index.min()]
min_date = df_min_date['Datum_dt_bckp'][0]
print(df_min_date)
values = {'last_survey_date': min_date}
df_survey_2_idx = df_survey_2_idx.fillna(value=values)

df_survey_2_idx

                 Datum  CDU_CSU   SPD  GRUENE  FDP  LINKE  PIRATEN  AfD  \
Datum_dt                                                                  
1998-01-08  01.08.1998     36.0  43.0     7.0  5.0    0.0      0.0  0.0   

            Linke_PDS  PDS  REP_DVU  Sonstige Befragte Zeitraum  \
Datum_dt                                                          
1998-01-08        0.0  4.0      3.0       2.0                     

           meta_insert_ts Datum_dt_bckp last_survey_date  
Datum_dt                                                  
1998-01-08                   1998-01-08              NaN  


,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts,Datum_dt_bckp,last_survey_date
Datum_dt,,,,,,,,,,,,,,,,,
1998-01-08,01.08.1998,36.0,43.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,,,,1998-01-08,1998-01-08
1998-02-05,02.05.1998,35.0,45.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,4.0,1.0,,,,1998-02-05,1998-01-08
1998-02-10,02.10.1998,36.0,41.0,7.0,6.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,,,,1998-02-10,1998-02-05
1998-04-04,04.04.1998,34.0,46.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,2.0,3.0,,,,1998-04-04,1998-02-10
1998-04-07,04.07.1998,35.0,44.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,,,,1998-04-07,1998-04-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-20,20.08.2020,37.0,16.0,17.0,6.0,8.0,0.0,10.0,0.0,0.0,0.0,6.0,1065,18.08.–19.08.,,2020-08-20,2020-07-23
2020-09-01,09.01.2020,27.0,13.0,23.0,9.0,8.0,0.0,14.0,0.0,0.0,0.0,6.0,1506,07.01.–08.01.,,2020-09-01,2020-08-20
2020-09-17,17.09.2020,36.0,16.0,20.0,5.0,8.0,0.0,10.0,0.0,0.0,0.0,5.0,1051,15.09.–16.09.,,2020-09-17,2020-09-01


In [174]:
df_survey_2_idx['datum_1'] = pd.to_datetime(df_survey_2_idx['last_survey_date'], format='%Y-%m-%d')
df_survey_2_idx['datum_2'] = pd.to_datetime(df_survey_2_idx['Datum_dt_bckp'], format='%Y-%m-%d')
df_survey_2_idx['nb_days_since_last_survey'] = df_survey_2_idx.apply(lambda x: (x['datum_2'] - x['datum_1']).days, axis=1)
# df_survey_2_idx['nb_days_since_last_survey'] = df_survey_2_idx['nb_days_since_last_survey'].astype(int)
df_survey_2_idx

,Datum,CDU_CSU,SPD,GRUENE,FDP,LINKE,PIRATEN,AfD,Linke_PDS,PDS,REP_DVU,Sonstige,Befragte,Zeitraum,meta_insert_ts,Datum_dt_bckp,last_survey_date,datum_1,datum_2,nb_days_since_last_survey
Datum_dt,,,,,,,,,,,,,,,,,,,,
1998-01-08,01.08.1998,36.0,43.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,,,,1998-01-08,1998-01-08,1998-01-08,1998-01-08,0
1998-02-05,02.05.1998,35.0,45.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,4.0,1.0,,,,1998-02-05,1998-01-08,1998-01-08,1998-02-05,28
1998-02-10,02.10.1998,36.0,41.0,7.0,6.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,,,,1998-02-10,1998-02-05,1998-02-05,1998-02-10,5
1998-04-04,04.04.1998,34.0,46.0,6.0,5.0,0.0,0.0,0.0,0.0,4.0,2.0,3.0,,,,1998-04-04,1998-02-10,1998-02-10,1998-04-04,53
1998-04-07,04.07.1998,35.0,44.0,7.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,,,,1998-04-07,1998-04-04,1998-04-04,1998-04-07,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-20,20.08.2020,37.0,16.0,17.0,6.0,8.0,0.0,10.0,0.0,0.0,0.0,6.0,1065,18.08.–19.08.,,2020-08-20,2020-07-23,2020-07-23,2020-08-20,28
2020-09-01,09.01.2020,27.0,13.0,23.0,9.0,8.0,0.0,14.0,0.0,0.0,0.0,6.0,1506,07.01.–08.01.,,2020-09-01,2020-08-20,2020-08-20,2020-09-01,12
2020-09-17,17.09.2020,36.0,16.0,20.0,5.0,8.0,0.0,10.0,0.0,0.0,0.0,5.0,1051,15.09.–16.09.,,2020-09-17,2020-09-01,2020-09-01,2020-09-17,16
